# Can we predict individual differences in behavior from brain activity?


Developed at Neurohackweek 2017

## Set up the environment and data

In [162]:
from HCPML_plt import clfAccHist
import os
import platform
import numpy as np
import nibabel as nib
import multiprocessing
import glob

In [180]:
# location of the data on disk
datapath = os.path.abspath('/Users/kevinsitek/Dropbox (MIT)/projects/nhw17/hcp_data_sample/')

In [181]:
# select the task and parameter estimate files to run
task = 'WM' # working memory task
pe_list = ['9','10']

Import the data:

In [183]:
sub_list=[] # subject index list
pe_bin_list=[] # list of (binarized) PEs

# add each subject to the data array
for sx, subpath in enumerate(glob.glob(datapath+'/*')):    
    print subpath
    # for each of the two contrasts
    for cx,pe in enumerate(pe_list):
        sub_list.append(sx+1) # save the subject index to a subject list
        pe_bin_list.append(cx) # save the (binarized) PE to a PE list
        
        # load the parameter estimates
        fpath = os.path.join(subpath, 'MNINonLinear/Results/tfMRI_%s/'%task,
                             'tfMRI_%s_hp200_s2_level2.feat/GrayordinatesStats/'%task,
                             'cope%s.feat/pe1.dtseries.nii'%(pe))
        pe1 = nib.load(fpath)
        
        # reshape the data into a 2-D array
        sub_pe_data = np.array(pe1.get_data().reshape(pe1.get_data().shape[4:]))
        
        # add the parameter estimate data to the overall data matrix
        if not sx and not cx:
            group_data = sub_pe_data
        else:
            group_data = np.concatenate((group_data, sub_pe_data),axis=0)

pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1


/Users/kevinsitek/Dropbox (MIT)/projects/nhw17/hcp_data_sample/105216
/Users/kevinsitek/Dropbox (MIT)/projects/nhw17/hcp_data_sample/105620
/Users/kevinsitek/Dropbox (MIT)/projects/nhw17/hcp_data_sample/105923
/Users/kevinsitek/Dropbox (MIT)/projects/nhw17/hcp_data_sample/106016
/Users/kevinsitek/Dropbox (MIT)/projects/nhw17/hcp_data_sample/106319
/Users/kevinsitek/Dropbox (MIT)/projects/nhw17/hcp_data_sample/106521
/Users/kevinsitek/Dropbox (MIT)/projects/nhw17/hcp_data_sample/106824
/Users/kevinsitek/Dropbox (MIT)/projects/nhw17/hcp_data_sample/107220


pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1


/Users/kevinsitek/Dropbox (MIT)/projects/nhw17/hcp_data_sample/107321
/Users/kevinsitek/Dropbox (MIT)/projects/nhw17/hcp_data_sample/107422


Check the data:

In [184]:
group_data.shape

(20, 91282)

In [185]:
np.array(pe_bin_list)

array([0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1])

In [186]:
print 'x shape: ', group_data.T.shape
print 'y shape: ', len(pe_bin_list)

x shape:  (91282, 20)
y shape:  20


## Run the classifier

In [320]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [327]:
# split the data for a master holdout test data set
def split_data(x, y):
    import numpy as np
    n_subs = len(y)/2
    rand_subs = np.random.choice(range(n_subs),n_subs,replace=False)
    
    rand_indices = []
    for i, r in enumerate(rand_subs):
        rand_indices.append(r*2)
        rand_indices.append(r*2+1)  
    Xtrain = x[:n_subs,:]
    Ytrain = y[:n_subs]
    Xtest = x[n_subs:,:]
    Ytest = y[n_subs:]
    return Xtrain, Xtest, Ytrain, Ytest, rand_indices

Xtrain, Xtest, Ytrain, Ytest, rand_indices = split_data(group_data, pe_bin_list)

In [328]:
rand_indices

[4, 5, 6, 7, 16, 17, 0, 1, 18, 19, 12, 13, 8, 9, 2, 3, 10, 11, 14, 15]

In [329]:
Xtrain.shape

(10, 91282)

In [330]:
# import the model
svc = SVC(kernel='linear')

In [331]:
# fit the model on the training data
svc.fit(Xtrain, Ytrain)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [332]:
# predict the labels of the test data
prediction = svc.predict(Xtest)
print prediction

[0 1 1 1 1 1 0 1 0 1]


In [333]:
# check the accuracy of the model's predictions
from sklearn.metrics import accuracy_score
print(accuracy_score(Ytest, prediction))

print "model accuracy: ", svc.score(Xtest, Ytest)

0.8
model accuracy:  0.8


In [ ]:
#convert feature weights to numpy array and save
sens_out = np.asarray(sens)
np.save(os.path.join(out_path,'cv_results',
                     '%dsubs_%s_CV_%s_ftrWghts'%(nsubs, cv_type, clf_type)),
        sens_out)

In [272]:
#feature weights x 2bk>0bk beta map
dp = np.zeros([len(subs_test),1])
for index, s in enumerate(subs_test):
    path = os.path.join(beta_path, s,
                                 'MNINonLinear', 'Results', 'tfMRI_'+task,
                                 'tfMRI_'+task+'_hp200_s2_level2.feat',
                                 'GrayordinatesStats','cope11.feat','pe1.dtseries.nii')
    beta_map  = nib.load(path)
    beta_map  = np.array(beta_map.dataobj)
    beta_map  = beta_map[0, 0, 0, 0, :, 0:]
    dp[index] = np.dot(sens_out,beta_map.transpose())

NameError: name 'subs_test' is not defined

In [ ]:
#load behavioral data
df    = pd.read_csv('HCP_behavioraldata.csv')
subs  = [int(s) for s in subs_test] #convert str to int
df2   = df.loc[df['Subject'].isin(subs_test)]
bdata = df2.ListSort_AgeAdj
bdata = bdata.reshape(30,1)


In [ ]:
# correlate predicted brain with behavioral measure
from scipy.stats import pearsonr
r, p = pearsonr(brain, behavior)